In [3]:
import numpy as np
import requests
import pandas as pd

In [4]:
#df = pd.read_csv("ClinicalTrialgraph_Journal_Analysis_March2020.csv")
df = pd.read_csv("total_pmid_citation_data_19thMay2020.csv")
#pmids = df['paper_pub_med_id'].unique()
pmids = df['pmid'].unique()

In [5]:
df.columns
len(pmids)

28398

In [6]:
tot_data = pd.read_csv("COCTR_multidimensional_ranking-master/ImportantDataDump/Final_Csv_Files_Having_All_Fields/FinalTotalData_17thMay2020.csv")

In [9]:
tot_data.shape

(43393, 2)

In [16]:
pubmed_id_list = list()
nct_id_list = list()

for row_id in range(tot_data.shape[0]):
    pm_many = tot_data.iloc[row_id,1].split(';')
    nct_many = [tot_data.iloc[row_id, 0] for i in range(len(pm_many))] 
    pubmed_id_list.extend(pm_many)
    nct_id_list.extend(nct_many)
    #print(pm_many, ' ; ', nct_many)

print(len(pubmed_id_list), len(nct_id_list))
pmid_cite_mapping = {'nct_id':nct_id_list, 'pmid_one':pubmed_id_list}
pmid_cite_mapping = pd.DataFrame(pmid_cite_mapping)
pmid_cite_mapping.to_csv('nct_pmid_citeonetoone_mapping_20thMay2020.csv', index=False)

64721 64721


In [8]:
final_pmids = list(set(pubmed_id_list) - set(pmids))
print(len(final_pmids))

41661


In [9]:
paper_pmid_list = []
ref_pmid_list = []
counter = 0

proxy_dict = {
    'http': 'http://172.16.2.30:8080',
    'https': 'https://172.16.2.30:8080',
    'ftp': 'ftp://172.16.2.30:8080'
}

for i in final_pmids:
    url = "https://www.ebi.ac.uk/europepmc/webservices/rest/MED/" + str(i) + "/references?page=1&pageSize=50&format=json"
    # r = requests.get('https://www.ebi.ac.uk/europepmc/webservices/rest/MED/i/references?page=1&pageSize=50&format=json')
    r = requests.get(url, proxies=proxy_dict)
    #print(r.status_code)
    data_json = r.json()

    #print(i)
    # pmid = data_json['request']['id']
    if data_json['hitCount']!=0:
        for j in data_json['referenceList']['reference']:
            if j['match']=='Y':
                paper_pmid_list.append(i)
                ref_pmid_list.append(j['id'])
    
    if counter % 100 == 0:
        print(counter)
    counter = counter + 1
    #print(ref_pmid_list)
    if counter % 500 == 0:
        print(len(paper_pmid_list), ' ; ', len(ref_pmid_list))
        pmid_cite_mapping = {'pmid':paper_pmid_list, 'ref_pmid':ref_pmid_list}
        pmid_cite_mapping = pd.DataFrame(pmid_cite_mapping)
        pmid_cite_mapping.to_csv('pmid_cite_mapping.csv', index=False)

0


KeyboardInterrupt: 